# Visualize MS3D

This is a notebook for visualizing detections on the point cloud. Here we show each step of the pseudo-labels such as KBF boxes, tracks, and final pseudo-labels so you can get an idea of how our approach works.

In [1]:
from quick_vis_bev import plot_boxes 
from pcdet.utils import compatibility_utils as compat
from pcdet.utils.transform_utils import ego_to_world, world_to_ego
from pcdet.utils.tracker_utils import get_frame_track_boxes

def load_pkl(pth):
    with open(pth, 'rb') as f:
        pkl_file = pickle.load(f)
    return pkl_file

def plot_pointcloud(pts, limit_range, size=0.5, c='black'):
    plt.figure(figsize=(8,8))
    ax = plt.subplot(111)    
    mask = mask_points_by_range(pts, limit_range)
    ax.scatter(pts[mask][:,0],pts[mask][:,1],s=size, c=c, marker='o')
    return plt, ax

all_dets = load_pkl('/OpenPCDet/tools/demo/demo_ps/all_dets.pkl') # Raw detections from all detectors
kbf_1f = load_pkl('/OpenPCDet/tools/demo/demo_ps/ps_dict_1f.pkl') # 1-frame KBF detections
kbf_16f = load_pkl('/OpenPCDet/tools/demo/demo_ps/ps_dict_16f.pkl') # 16-frame KBF detections
tracks_1f = load_pkl('/OpenPCDet/tools/demo/demo_ps/tracks_1f.pkl')
tracks_16f = load_pkl('/OpenPCDet/tools/demo/demo_ps/tracks_16f.pkl')
ps_e0 = load_pkl('/OpenPCDet/tools/demo/demo_ps/ps_label_e0.pkl') # Final pseudo-labels

SyntaxError: invalid syntax (2850252348.py, line 25)

In [ ]:
%matplotlib notebook

idx = 20
pc_range = 80

## Note: uncomment all ego_to_world lines if you want to visualize in world frame

# ===== Plot pointcloud =====
frame_id = compat.get_frame_id(target_set, target_set.infos[idx])
pose = compat.get_pose(target_set, frame_id)
pts = target_set[idx]['points']
# pts, _ = ego_to_world(pose, points=pts) # Uncomment if you want to visualize in world frame
pc_mean = np.mean(pts, axis=0)    
limit_range = [pc_mean[0]-pc_range, pc_mean[1]-pc_range, -5.0, pc_mean[0]+pc_range, pc_mean[1]+pc_range, 3.0]
plt, ax = plot_pointcloud(pts, limit_range, size=0.5, c=np.array([[0.3,0.3,0.3]]))

# ===== Plot ground truth boxes =====
classes=['Vehicle']
class_mask = np.array([n in classes for n in compat.get_gt_names(target_set, frame_id)], 
                      dtype=np.bool_)
gt_boxes = compat.get_gt_boxes(target_set, frame_id)[class_mask]
# _, gt_boxes = ego_to_world(pose, boxes=gt_boxes) 
plot_boxes(ax, gt_boxes, color=[0.2,0.3,0.8],linewidth=2,label='Ground-truth', limit_range=limit_range, alpha=1)

# ===== Plot KBF 1-frame =====
dets = kbf_1f[frame_id]['gt_boxes']
score_mask = dets[:,8] > 0.4
# _, ps = ego_to_world(pose, boxes=ps)
plot_boxes(ax, dets[score_mask], scores=dets[:,8][score_mask],
               color=[0.8,0,0.8],linestyle='solid',linewidth=1.5,label='KBF-1F',
               limit_range=limit_range, alpha=1)

# ===== Plot KBF 16-frame =====
dets = kbf_16f[frame_id]['gt_boxes']
score_mask = dets[:,8] > 0.4
# _, ps = ego_to_world(pose, boxes=ps)
plot_boxes(ax, dets[score_mask], scores=dets[:,8][score_mask],
               color=[0.8,0,0.8],linestyle='solid',linewidth=1.5,label='KBF-16F',
               limit_range=limit_range, alpha=1)

# ===== Plot 1-frame tracks =====
# Tracks are in world frame; need to transform to ego frame to visualize with the rest of the boxes
tboxes = get_frame_track_boxes(tracks_1f, frame_id, frame2box_key='frameid_to_box')
_, tboxes = world_to_ego(pose, boxes=tboxes) 
plot_boxes(ax, tboxes, 
               scores=tboxes[:,8],
               color=[0.7,0,0.7],
               linestyle='solid',linewidth=2.0,
               limit_range=limit_range, alpha=1)

# ===== Plot 16-frame tracks =====
tboxes = get_frame_track_boxes(tracks_16f, frame_id, frame2box_key='frameid_to_box')
_, tboxes = world_to_ego(pose, boxes=tboxes)
plot_boxes(ax, tboxes, 
               scores=tboxes[:,8],
               color=[0.7,0,0.7],
               linestyle='solid',linewidth=2.0,
               limit_range=limit_range, alpha=1)

# ===== Plot final pseudo labels =====
dets = ps_e0[frame_id]['gt_boxes']
score_mask = dets[:,8] > 0.4
# _, ps = ego_to_world(pose, boxes=ps)
plot_boxes(ax, dets[score_mask], scores=dets[:,8][score_mask],
               color=[0.8,0,0.8],linestyle='solid',linewidth=1.5,label='Final Labels',
               limit_range=limit_range, alpha=1)